# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 30 2023 11:53AM,256190,20,9094502,"Exact-Rx, Inc.",Released
1,Jan 30 2023 11:53AM,256186,19,ADV80011062,"AdvaGen Pharma, Ltd",Released
2,Jan 30 2023 11:53AM,256186,19,ADV80011063,"AdvaGen Pharma, Ltd",Released
3,Jan 30 2023 11:53AM,256186,19,ADV80011064,"AdvaGen Pharma, Ltd",Released
4,Jan 30 2023 11:53AM,256186,19,ADV80011065,"AdvaGen Pharma, Ltd",Released
5,Jan 30 2023 11:53AM,256186,19,ADV80011066,"AdvaGen Pharma, Ltd",Released
6,Jan 30 2023 11:53AM,256186,19,ADV80011068,"AdvaGen Pharma, Ltd",Released
7,Jan 30 2023 11:53AM,256186,19,ADV80011069,"AdvaGen Pharma, Ltd",Released
8,Jan 30 2023 11:53AM,256186,19,ADV80011070,"AdvaGen Pharma, Ltd",Released
9,Jan 30 2023 11:53AM,256186,19,ADV80011071,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,256185,Released,1
24,256186,Released,55
25,256188,Released,18
26,256189,Released,1
27,256190,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256185,NaN,NaN,1.0
256186,NaN,NaN,55.0
256188,NaN,NaN,18.0
256189,NaN,NaN,1.0
256190,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256139,2.0,0.0,11.0
256140,0.0,1.0,0.0
256145,0.0,0.0,28.0
256148,0.0,0.0,52.0
256151,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256139,2,0,11
256140,0,1,0
256145,0,0,28
256148,0,0,52
256151,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256139,2,0,11
1,256140,0,1,0
2,256145,0,0,28
3,256148,0,0,52
4,256151,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256139,2,,11
1,256140,,1,
2,256145,,,28
3,256148,,,52
4,256151,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 30 2023 11:53AM,256190,20,"Exact-Rx, Inc."
1,Jan 30 2023 11:53AM,256186,19,"AdvaGen Pharma, Ltd"
56,Jan 30 2023 11:53AM,256189,20,"Exact-Rx, Inc."
57,Jan 30 2023 11:52AM,256188,20,"Exact-Rx, Inc."
75,Jan 30 2023 11:41AM,256185,16,"Emersa Waterbox, LLC"
76,Jan 30 2023 11:27AM,256183,15,DXM Pharmaceutical Inc.
77,Jan 30 2023 11:27AM,256182,15,"Emersa Waterbox, LLC"
78,Jan 30 2023 11:23AM,256181,15,Lonza
79,Jan 30 2023 10:44AM,256178,12,"SD Head USA, LLC"
80,Jan 30 2023 10:41AM,256177,20,"HVL, LLC dba Atrium Innovations"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 30 2023 11:53AM,256190,20,"Exact-Rx, Inc.",,,1
1,Jan 30 2023 11:53AM,256186,19,"AdvaGen Pharma, Ltd",,,55
2,Jan 30 2023 11:53AM,256189,20,"Exact-Rx, Inc.",,,1
3,Jan 30 2023 11:52AM,256188,20,"Exact-Rx, Inc.",,,18
4,Jan 30 2023 11:41AM,256185,16,"Emersa Waterbox, LLC",,,1
5,Jan 30 2023 11:27AM,256183,15,DXM Pharmaceutical Inc.,,,1
6,Jan 30 2023 11:27AM,256182,15,"Emersa Waterbox, LLC",,,1
7,Jan 30 2023 11:23AM,256181,15,Lonza,,,1
8,Jan 30 2023 10:44AM,256178,12,"SD Head USA, LLC",,,1
9,Jan 30 2023 10:41AM,256177,20,"HVL, LLC dba Atrium Innovations",,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 30 2023 11:53AM,256190,20,"Exact-Rx, Inc.",1,,
1,Jan 30 2023 11:53AM,256186,19,"AdvaGen Pharma, Ltd",55,,
2,Jan 30 2023 11:53AM,256189,20,"Exact-Rx, Inc.",1,,
3,Jan 30 2023 11:52AM,256188,20,"Exact-Rx, Inc.",18,,
4,Jan 30 2023 11:41AM,256185,16,"Emersa Waterbox, LLC",1,,
5,Jan 30 2023 11:27AM,256183,15,DXM Pharmaceutical Inc.,1,,
6,Jan 30 2023 11:27AM,256182,15,"Emersa Waterbox, LLC",1,,
7,Jan 30 2023 11:23AM,256181,15,Lonza,1,,
8,Jan 30 2023 10:44AM,256178,12,"SD Head USA, LLC",1,,
9,Jan 30 2023 10:41AM,256177,20,"HVL, LLC dba Atrium Innovations",,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 30 2023 11:53AM,256190,20,"Exact-Rx, Inc.",1,,
1,Jan 30 2023 11:53AM,256186,19,"AdvaGen Pharma, Ltd",55,,
2,Jan 30 2023 11:53AM,256189,20,"Exact-Rx, Inc.",1,,
3,Jan 30 2023 11:52AM,256188,20,"Exact-Rx, Inc.",18,,
4,Jan 30 2023 11:41AM,256185,16,"Emersa Waterbox, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 30 2023 11:53AM,256190,20,"Exact-Rx, Inc.",1.0,NaN,NaN
1,Jan 30 2023 11:53AM,256186,19,"AdvaGen Pharma, Ltd",55.0,NaN,NaN
2,Jan 30 2023 11:53AM,256189,20,"Exact-Rx, Inc.",1.0,NaN,NaN
3,Jan 30 2023 11:52AM,256188,20,"Exact-Rx, Inc.",18.0,NaN,NaN
4,Jan 30 2023 11:41AM,256185,16,"Emersa Waterbox, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 30 2023 11:53AM,256190,20,"Exact-Rx, Inc.",1.0,0.0,0.0
1,Jan 30 2023 11:53AM,256186,19,"AdvaGen Pharma, Ltd",55.0,0.0,0.0
2,Jan 30 2023 11:53AM,256189,20,"Exact-Rx, Inc.",1.0,0.0,0.0
3,Jan 30 2023 11:52AM,256188,20,"Exact-Rx, Inc.",18.0,0.0,0.0
4,Jan 30 2023 11:41AM,256185,16,"Emersa Waterbox, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2561599,213.0,20.0,2.0
12,512318,1.0,1.0,0.0
15,768546,3.0,0.0,0.0
16,1024642,4.0,0.0,0.0
19,256186,55.0,0.0,0.0
20,1024744,20.0,1.0,0.0
21,256172,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2561599,213.0,20.0,2.0
1,12,512318,1.0,1.0,0.0
2,15,768546,3.0,0.0,0.0
3,16,1024642,4.0,0.0,0.0
4,19,256186,55.0,0.0,0.0
5,20,1024744,20.0,1.0,0.0
6,21,256172,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,213.0,20.0,2.0
1,12,1.0,1.0,0.0
2,15,3.0,0.0,0.0
3,16,4.0,0.0,0.0
4,19,55.0,0.0,0.0
5,20,20.0,1.0,0.0
6,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,213.0
1,12,Released,1.0
2,15,Released,3.0
3,16,Released,4.0
4,19,Released,55.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,2.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,20.0,1.0,0.0,0.0,0.0,1.0,0.0
Released,213.0,1.0,3.0,4.0,55.0,20.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,20.0,1.0,0.0,0.0,0.0,1.0,0.0
2,Released,213.0,1.0,3.0,4.0,55.0,20.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,20.0,1.0,0.0,0.0,0.0,1.0,0.0
2,Released,213.0,1.0,3.0,4.0,55.0,20.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()